In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions._


spark: SparkSession = org.apache.spark.sql.SparkSession@e166017

In [2]:
import spark.implicits._

case class Employee(employee_id: Int, name: String, manager_id: Option[Int], salary: Int)

val employee: Seq[(Int, String, Option[Int], Int)] = Seq(
  (3, "Mila", Some(9), 60301),
  (12, "Antonella", None, 31000),
  (13, "Emery", None, 67084),
  (1, "Kalel", Some(11), 21241),
  (9, "Mikaela", None, 50937),
  (11, "Joziah", Some(6), 28485)
)

val employeesDS = employee.toDF("employee_id", "name", "manager_id", "salary").as[Employee]

import spark.implicits._


defined class Employee
employee: Seq[(Int, String, Option[Int], Int)] = List(
  (3, "Mila", Some(value = 9), 60301),
  (12, "Antonella", None, 31000),
  (13, "Emery", None, 67084),
  (1, "Kalel", Some(value = 11), 21241),
  (9, "Mikaela", None, 50937),
  (11, "Joziah", Some(value = 6), 28485)
)
employeesDS: Dataset[Employee] = [employee_id: int, name: string ... 2 more fields]

Find the IDs of the employees whose salary is strictly less than $30000 and whose manager left the company. When a manager leaves the company, their information is deleted from the Employees table, but the reports still have their manager_id set to the manager that left.

Return the result table ordered by employee_id.

The result format is in the following example.

| employee_id |
|-------------|
| 11          |

In [3]:
employeesDS
    .filter(col("salary") < lit(30000))
    .as("a").join(
        employeesDS.as("b"), 
        col("a.manager_id") === col("b.employee_id"),
        "left_outer"
    )
    .filter((col("b.name").isNull) && (col("a.manager_id").isNotNull))
    .select(
        col("a.employee_id").as("employee_id")
    )
    .show()

+-----------+
|employee_id|
+-----------+
|         11|
+-----------+



In [4]:
spark.stop()